# Step 0 - Importing Libraries

### Notes
- In the future I should consider making a dataframe that has the individual songs lyrics, their timestamp, title, and other stats.

In [1]:
# !pip install -U contractions
# !pip install -U inflect
# !pip install -U nltk

In [1]:
import re, string, unicodedata
import nltk
import contractions
import codecs
import inflect
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize, sent_tokenize
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vonn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vonn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vonn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Step 1 read in data

In [2]:
# Read in data
with codecs.open('Raw_Dataframe','rb') as f:
    df_1 = pickle.load(f)

df_1.set_index('Artist',inplace = True)

In [3]:
df_1

,Lyrics
Artist,
Drake,"[Produced by Boi-1da, Frank Dukes, Noah ""40"" S..."
Jayz,[Intro: Hannah Williams]\nDo I find it so hard...
Nas,[Produced by Ron Browz]\n\n[Intro]\nFuck Jay Z...
Eminem,"[Verse 1]\nNow this shit's about to kick off, ..."
Future,[Intro]\nHigh Klassified な音楽\nI got the truth ...
KanyeWest,[Produced By Daft Punk & Kanye West]\n\n[Verse...


In [4]:
df_1.loc['Drake'][0][0:100]

'[Produced by Boi-1da, Frank Dukes, Noah "40" Shebib, & Nineteen85]\n\n[Part I: 0 to 100]\n\n[Verse 1]\nFu'

In [5]:
def first_cleaning_session():
    """
    I am creating a function to do the first cleaning session.
    There are simple things that will show up in any version of the Lyrics
    that I obtain. This function will handle it before moving on to harder
    cleaning session regarding lemmatizing, stemming, or profanity.
    """
    index = list(df_1.index.values)
    rapper = df_1.Lyrics
    
    for artist_ in index:
        # Lower text
        try:
            rapper.loc[artist_] = rapper.loc[artist_].lower();
        except:
            pass
        # remove '\' from text that is in between apostrophes
#         try:
#             rapper.loc[artist_] = rapper.loc[artist_].replace("\\", "");
#         except:
#             pass
        # Remove punctuation
        try:
            rapper.loc[artist_] = re.sub("[^\w\d\s]+", "", rapper.loc[artist_])
        except:
            pass
        # Split text
        try:
            rapper.loc[artist_] = rapper.loc[artist_].split("\n");
        except:
            pass
        # remove producer
        try:
            rapper.loc[artist_] = [line for line in df_1.loc[artist_][0] if ('[produced' not in line) == True]
        except:
            pass
        # remove blank strings
        try:
            rapper.loc[artist_] = list(filter(None, df_1.loc[artist_][0]))
        except:
            pass
        

In [6]:
first_cleaning_session()

In [7]:
df_1

,Lyrics
Artist,
Drake,[produced by boi1da frank dukes noah 40 shebib...
Jayz,"[intro hannah williams, do i find it so hard, ..."
Nas,"[produced by ron browz, intro, fuck jay z, wha..."
Eminem,"[verse 1, now this shits about to kick off thi..."
Future,"[intro, high klassified な音楽, i got the truth i..."
KanyeWest,"[produced by daft punk kanye west, verse 1, f..."


In [8]:
df_1.loc['Drake'][0]
# now it is a list of lyrics

['produced by boi1da frank dukes noah 40 shebib  nineteen85',
 'part i 0 to 100',
 'verse 1',
 'fuck bein on some chill shit',
 'we go 0 to 100 nigga real quick',
 'they be on that raptopaythebill shit',
 'and i dont feel that shit not even a little bit',
 'oh lord know yourself know your worth nigga',
 'my actions been louder than my words nigga',
 'how you so high but still so down to earth nigga',
 'if niggas wanna do it we can do it on they turf nigga',
 'oh lord im the rookie and the vet',
 'shoutout to the bitches out here holdin down the set',
 'all up in my phone lookin at pictures from the other night',
 'she gon be upset if she keep scrollin to the left dawg',
 'she gon see some shit that she dont wanna see',
 'she aint ready for it',
 'if i aint the greatest then im headed for it',
 'that mean im way up way up',
 'yeah the 6 aint friendly but thats where i lay up',
 'this shit a mothafuckin lay up',
 'i been steph curry with the shot',
 'been cookin with the sauce chef curry

In [9]:
df_1

,Lyrics
Artist,
Drake,[produced by boi1da frank dukes noah 40 shebib...
Jayz,"[intro hannah williams, do i find it so hard, ..."
Nas,"[produced by ron browz, intro, fuck jay z, wha..."
Eminem,"[verse 1, now this shits about to kick off thi..."
Future,"[intro, high klassified な音楽, i got the truth i..."
KanyeWest,"[produced by daft punk kanye west, verse 1, f..."


In [10]:
df_1.loc['Drake'][0][0:10]

['produced by boi1da frank dukes noah 40 shebib  nineteen85',
 'part i 0 to 100',
 'verse 1',
 'fuck bein on some chill shit',
 'we go 0 to 100 nigga real quick',
 'they be on that raptopaythebill shit',
 'and i dont feel that shit not even a little bit',
 'oh lord know yourself know your worth nigga',
 'my actions been louder than my words nigga',
 'how you so high but still so down to earth nigga']

In [11]:
song = {}
index = list(df_1.index.values)
for artst in index:
    var_1 = df_1.Lyrics.loc[artst]
    song[artst] = []
    s = " "
    s = s.join(var_1)
    lst = re.split('\[(.*?)\]', s)
    for i in lst:
        if ":" in i:
            if artst in i:
                song[artst].append(i)
        else:
            song[artst].append(i)

In [12]:
for artst in index:
    print(f'{artst}')
    rapper = df_1.Lyrics
    print(f'{df_1.Lyrics}')
    print(f'song[artst]')
    rapper.loc[artst] = song[artst]

df_1

Drake
Artist
Drake        [produced by boi1da frank dukes noah 40 shebib...
Jayz         [intro hannah williams, do i find it so hard, ...
Nas          [produced by ron browz, intro, fuck jay z, wha...
Eminem       [verse 1, now this shits about to kick off thi...
Future       [intro, high klassified な音楽, i got the truth i...
KanyeWest    [produced by daft punk  kanye west, verse 1, f...
Name: Lyrics, dtype: object
song[artst]
Jayz
Artist
Drake        [produced by boi1da frank dukes noah 40 shebib...
Jayz         [intro hannah williams, do i find it so hard, ...
Nas          [produced by ron browz, intro, fuck jay z, wha...
Eminem       [verse 1, now this shits about to kick off thi...
Future       [intro, high klassified な音楽, i got the truth i...
KanyeWest    [produced by daft punk  kanye west, verse 1, f...
Name: Lyrics, dtype: object
song[artst]
Nas
Artist
Drake        [produced by boi1da frank dukes noah 40 shebib...
Jayz         [intro hannah williams do i find it so hard wh...
Na

,Lyrics
Artist,
Drake,[produced by boi1da frank dukes noah 40 shebib...
Jayz,[intro hannah williams do i find it so hard wh...
Nas,[produced by ron browz intro fuck jay z whats ...
Eminem,[verse 1 now this shits about to kick off this...
Future,[intro high klassified な音楽 i got the truth in ...
KanyeWest,[produced by daft punk kanye west verse 1 for...


In [13]:
song = {}
index = list(df_1.index.values)
for artst in index:
    song[artst] = []
    print(f'{artst}')
    lst = df_1.loc[artst][0]
    for i in lst:
        if len(i) > 20:
            song[artst].append(i)

Drake
Jayz
Nas
Eminem
Future
KanyeWest


In [14]:
for artst in index:
    print(f'{artst}')
    rapper = df_1.Lyrics
    print(f'{df_1.Lyrics}')
    print(f'song[artst]')
    rapper.loc[artst] = song[artst]

df_1

Drake
Artist
Drake        [produced by boi1da frank dukes noah 40 shebib...
Jayz         [intro hannah williams do i find it so hard wh...
Nas          [produced by ron browz intro fuck jay z whats ...
Eminem       [verse 1 now this shits about to kick off this...
Future       [intro high klassified な音楽 i got the truth in ...
KanyeWest    [produced by daft punk  kanye west verse 1 for...
Name: Lyrics, dtype: object
song[artst]
Jayz
Artist
Drake        [produced by boi1da frank dukes noah 40 shebib...
Jayz         [intro hannah williams do i find it so hard wh...
Nas          [produced by ron browz intro fuck jay z whats ...
Eminem       [verse 1 now this shits about to kick off this...
Future       [intro high klassified な音楽 i got the truth in ...
KanyeWest    [produced by daft punk  kanye west verse 1 for...
Name: Lyrics, dtype: object
song[artst]
Nas
Artist
Drake        [produced by boi1da frank dukes noah 40 shebib...
Jayz         [intro hannah williams do i find it so hard wh...
Na

,Lyrics
Artist,
Drake,[produced by boi1da frank dukes noah 40 shebib...
Jayz,[intro hannah williams do i find it so hard wh...
Nas,[produced by ron browz intro fuck jay z whats ...
Eminem,[verse 1 now this shits about to kick off this...
Future,[intro high klassified な音楽 i got the truth in ...
KanyeWest,[produced by daft punk kanye west verse 1 for...


In [15]:
first_cleaning_session()

In [16]:
df_1

,Lyrics
Artist,
Drake,[produced by boi1da frank dukes noah 40 shebib...
Jayz,[intro hannah williams do i find it so hard wh...
Nas,[produced by ron browz intro fuck jay z whats ...
Eminem,[verse 1 now this shits about to kick off this...
Future,[intro high klassified な音楽 i got the truth in ...
KanyeWest,[produced by daft punk kanye west verse 1 for...


In [17]:
index = list(df_1.index.values)
rapper = df_1.Lyrics

s = " "
for artist_ in index:
    rapper.loc[artist_] = s.join(rapper[artist_])

### Corpus

In [18]:
# After cleaning this dataframe is in the form of a corpus.
df_1

,Lyrics
Artist,
Drake,produced by boi1da frank dukes noah 40 shebib ...
Jayz,intro hannah williams do i find it so hard whe...
Nas,produced by ron browz intro fuck jay z whats u...
Eminem,verse 1 now this shits about to kick off this ...
Future,intro high klassified な音楽 i got the truth in m...
KanyeWest,produced by daft punk kanye west verse 1 for ...


In [19]:
# For visualizations later on let's create a column of the artists' names. 
full_names = df_1.index.tolist()
df_1['Artist Name'] = full_names
df_1

,Lyrics,Artist Name
Artist,,
Drake,produced by boi1da frank dukes noah 40 shebib ...,Drake
Jayz,intro hannah williams do i find it so hard whe...,Jayz
Nas,produced by ron browz intro fuck jay z whats u...,Nas
Eminem,verse 1 now this shits about to kick off this ...,Eminem
Future,intro high klassified な音楽 i got the truth in m...,Future
KanyeWest,produced by daft punk kanye west verse 1 for ...,KanyeWest


In [20]:
#pickle it for later use
df_1.to_pickle('DataFrame_Corpus.pkl')

### Countvectorizer

In [22]:
stop_words = set(stopwords.words("english"))
cv = CountVectorizer(stop_words=stop_words)
df_cv = cv.fit_transform(df_1.Lyrics)
df_dtm = pd.DataFrame(df_cv.toarray(), columns=cv.get_feature_names())
df_dtm.index = df_1.index
df_dtm

,02,10,100,1000,1008,10yearolds,11,12,125,140,...,zeros,zip,zod,zombie,zone,zonin,zöld,ölén,úgy,な音楽
Artist,,,,,,,,,,,,,,,,,,,,,
Drake,0,0,6,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
Jayz,0,0,2,0,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Nas,0,1,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
Eminem,1,0,0,0,0,1,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
Future,0,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
KanyeWest,0,0,0,0,0,0,0,1,2,0,...,0,0,0,1,0,3,1,1,1,0


In [23]:
# Let's pickle it for later use
df_dtm.to_pickle('DataFrame_Document_Term_Matrix.pkl')

# Step 3 Tokenize

In [24]:
index = list(df_1.index.values)
rapper = df_1.Lyrics
for artist_ in index:
    rapper.loc[artist_] = [nltk.word_tokenize(rapper.loc[artist_][i]) for i in range(len(rapper.loc[artist_]))]

df_1

,Lyrics,Artist Name
Artist,,
Drake,"[[p], [r], [o], [d], [u], [c], [e], [d], [], [...",Drake
Jayz,"[[i], [n], [t], [r], [o], [], [h], [a], [n], [...",Jayz
Nas,"[[p], [r], [o], [d], [u], [c], [e], [d], [], [...",Nas
Eminem,"[[v], [e], [r], [s], [e], [], [1], [], [n], [o...",Eminem
Future,"[[i], [n], [t], [r], [o], [], [h], [i], [g], [...",Future
KanyeWest,"[[p], [r], [o], [d], [u], [c], [e], [d], [], [...",KanyeWest


# Add'l cleaning functions

Credit due to KDNuggets' Text Data Preprocessing Walkthrough

In [25]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '': # we don't want to append empty spaces now that we have refined list with out punctuation.
            new_words.append(new_word)
    return new_words

In [26]:
def replace_numbers_with_their_text_form(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit(): # it can recognize digits and if it is...
            new_word = p.number_to_words(word) # converts it to words
            new_words.append(new_word) # then appends
        else:
            new_words.append(word) # if not it just appends
    return new_words #after all of that it returns the lsit

In [27]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'): # if it doesn't exist in the stop words list...
            new_words.append(word)
    return new_words

In [28]:
def stemming_the_words(words):
    """stem the words. As in turn them into their dictionary root word.
    Lancaster Stemmer is apparently extremely aggressive in cutting down
    your words so I read that porter is subtle but fast, so going to switch to Porter2
    also known as snowball."""
    new_words = []
    for word in words:
        stemmer = SnowballStemmer('english') # don't need to do it cause already done above but you can ignore stopwords here.
        stems =[]
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
    return stems

In [29]:
def lemmatize_verbs(words):
    """Lemmatize just the verbs in list of tokenized words. 
    Don't know if they were stemmed in the previous function. Would love
    to know what happens to them prior to lemmatizing. Also would it be 
    the same if I did this with nouns and adjectives"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [30]:
# Apply the functions to the tokenized list
def normalize(words):
    words = remove_punctuation(words)                     
    words = remove_stopwords(words)                     
    words = replace_numbers_with_their_text_form(words)   
    return words

In [31]:
for artst in index:
    normalize(df_1.Lyrics[artst])

TypeError: expected string or bytes-like object

In [ ]:
df_1

In [ ]:
# Stemmed and Lemmatized words
def stem_and_lemmatized_words(words):
    lemma = lemmatize_verbs    (words)  
    stem  = stemming_the_words (words)
    return stem, lemma

In [ ]:
stem, lemma = stem_and_lemmatized_words(words)
print('Stemmed:\n\n', stem)
print()
print('Lemmatized:\n\n', lemma)

# Save to File

In [ ]:
with open('lemmatized_list','wb') as f:
    pickle.dump(lemma, f)

In [ ]:
with open('stemmed_list', 'wb') as f:
    pickle.dump(stem, f)

In [ ]:
with open('corpus_list_raw', 'wb') as f:
    pickle.dump(corpus, f)

In [ ]:
# see if they saved okay
with open('lemmatized_list', 'rb') as f:
    lemma = pickle.load(f)

In [ ]:
lemma

In [ ]:
with open('stemmed_list', 'rb') as f:
    stem = pickle.load(f)

stem

In [ ]:
with open ('corpus_list_raw', 'rb') as f:
    corpus = pickle.load(f)
    
corpus